In [38]:
LABEL_DIR = "/home/lab530/KenYu/visualDet3D/kitti/training/label_2/"
PREDI_DIR = "/home/lab530/KenYu/visualDet3D/my_exp/Mono3D/output/validation/data_rename/"
# 
# LABEL_DIR = "/home/lab530/KenYu/little_exp/test_label/"
# PREDI_DIR = "/home/lab530/KenYu/little_exp/test_predi/"
IOU_THRES = 0.7

def bbox3DIOU(b1, b2):
    '''
    b1 = [(x1, x2), (y1, y2), (z1, z2)]
    '''
    inter_xyz = []
    for i in range(3):
        if b1[i][0] < b2[i][0]:
            bmin = (b1[i][0], b1[i][1])
            bmax = (b2[i][0], b2[i][1])
        else:
            bmin = (b2[i][0], b2[i][1])
            bmax = (b1[i][0], b1[i][1])
        
        if bmax[0] <= bmin[1]: # inter_x
            inter_xyz.append(bmin[1] - bmax[0])
        else:
            inter_xyz.append(0.0)
    
    inter_vol = inter_xyz[0]*inter_xyz[1]*inter_xyz[2]
    b1_vol = (b1[0][1] - b1[0][0]) * (b1[1][1] - b1[1][0]) * (b1[2][1] - b1[2][0])
    b2_vol = (b2[0][1] - b2[0][0]) * (b2[1][1] - b2[1][0]) * (b1[2][1] - b2[2][0])

    return inter_vol / (b1_vol + b2_vol - inter_vol) # Intersetion over union

import numpy as np
import os
from collections import defaultdict

pred_fns = os.listdir(PREDI_DIR)
print(f"Number of prediction = {pred_fns.__len__()}")

tp = 0
fp = 0
fn = 0

fn_truncated = defaultdict(int)
fn_occluded = defaultdict(int)

for file_name in pred_fns:
    b_gts = []
    b_gts_info = []
    with open(LABEL_DIR + file_name , 'r') as f_gt:
        for line in f_gt.readlines():
            det = line.strip().split(" ")
            cls = det[0]
            if cls != "Car": continue
            truncated, occluded, alpha, x1,y1,x2,y2,  h,w,l,  x,y,z, rotation_y = [ float(i) for i in det[1:] ]
            b_gts.append( [(x-w/2, x+w/2), (y-h, y), (z-l/2, z+l/2)] )
            b_gts_info.append( (cls, truncated, occluded, alpha, x1,y1,x2,y2,  h,w,l,  x,y,z, rotation_y) )
    
    with open(PREDI_DIR + file_name , 'r') as f_pred:
        for line in f_pred.readlines():
            det = line.strip().split(" ")
            cls = det[0]
            if cls != "Car": continue
            truncated, occluded, alpha, x1,y1,x2,y2,  h,w,l,  x,y,z, rotation_y, score = [ float(i) for i in det[1:] ]
            b_pred = [(x-w/2, x+w/2), (y-h, y), (z-l/2, z+l/2)]

            iou_list = []
            for b_gt in b_gts:
                iou_list.append( bbox3DIOU(b_pred, b_gt) )
            iou_list = np.array(iou_list)
            
            if len(iou_list) != 0 and max(iou_list) >= IOU_THRES: # this is a true positive detection 
                max_idx = np.argmax(iou_list)
                tp += 1
                del b_gts[max_idx]
                del b_gts_info[max_idx]
            else:
                fp += 1
    fn += len(b_gts)
    # Find reason of its false negative
    for b_gt_info in b_gts_info:
        fn_truncated[b_gt_info[1]//0.1] += 1
        fn_occluded[b_gt_info[2]] += 1

print(f"tp = {tp}")
print(f"fp = {fp}")
print(f"fn = {fn}")

# 
print("============== False Nagative Reasons =================")
for i in sorted(fn_occluded):
    print(f"occluded level_{i}: {fn_occluded[i]}  ({round(100*fn_occluded[i]/fn,2)}%)")
for i in sorted(fn_truncated):
    print(f"Truncated {round(i*10, 2)}% : {fn_truncated[i]}  ({round(100*fn_truncated[i]/fn,2)}%)")

# TODO object location heatmap 
# TODO Too Far object? 


Number of prediction = 3769
tp = 3414
fp = 5955
fn = 10971
============== False Nagative Reasons =================
occluded level_0.0: 4583  (41.77%)
occluded level_1.0: 2957  (26.95%)
occluded level_2.0: 2966  (27.03%)
occluded level_3.0: 465  (4.24%)
Truncated 0.0% : 9365  (85.36%)
Truncated 10.0% : 100  (0.91%)
Truncated 20.0% : 125  (1.14%)
Truncated 30.0% : 80  (0.73%)
Truncated 40.0% : 126  (1.15%)
Truncated 50.0% : 115  (1.05%)
Truncated 60.0% : 142  (1.29%)
Truncated 70.0% : 155  (1.41%)
Truncated 80.0% : 261  (2.38%)
Truncated 90.0% : 502  (4.58%)
